In [1]:
# !pip3 uninstall -y sparqlwrapper
# !pip3 install sparqlwrapper

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\creeg\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [25]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [3]:
#Declare the BNB endpoint
endpoint="http://bnb.data.bl.uk/sparql"
sparql = SPARQLWrapper(endpoint)

In [4]:
# test 

q='''PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX bio: <http://purl.org/vocab/bio/0.1/>
PREFIX blt: <http://www.bl.uk/schemas/bibliographic/blterms#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX isbd: <http://iflastandards.info/ns/isbd/elements/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rda: <http://rdvocab.info/ElementsGr2/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?book ?bnb ?title WHERE {
    #Match the book by ISBN
    ?book bibo:isbn13 "9780729408745";
    #bind some variables to its other attributes
    blt:bnb ?bnb;
    dct:title ?title. }'''
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results

{'head': {'link': [], 'vars': ['book', 'bnb', 'title']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'book': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/resource/005711109'},
    'bnb': {'type': 'literal', 'value': 'GBA514449'},
    'title': {'type': 'literal',
     'value': 'The complete works of Voltaire = Les œuvres completes de Voltaire'}}]}}

In [5]:
# create template

prefix='''
PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX bio: <http://purl.org/vocab/bio/0.1/>
PREFIX blt: <http://www.bl.uk/schemas/bibliographic/blterms#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX isbd: <http://iflastandards.info/ns/isbd/elements/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rda: <http://rdvocab.info/ElementsGr2/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    
'''

In [15]:
# Search for books by author name
q='''
SELECT DISTINCT ?book ?title ?author ?genre WHERE {
    ?book dct:creator ?author ;
        dct:title ?title.
    ?author foaf:name "Ernest Hemingway".
} LIMIT 5
'''

In [17]:
# Run query, parse as JSON, place into variable

sparql.setQuery(prefix+q)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [18]:
results

{'head': {'link': [], 'vars': ['book', 'title', 'author', 'genre']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'book': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/resource/019676998'},
    'title': {'type': 'literal',
     'value': 'The good life according to Hemingway'},
    'author': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/person/HemingwayErnest1899-1961'}},
   {'book': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/resource/006731265'},
    'title': {'type': 'literal', 'value': 'The essential Hemingway'},
    'author': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/person/HemingwayErnest1899-1961'}},
   {'book': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/resource/006969792'},
    'title': {'type': 'literal', 'value': 'Death in the afternoon'},
    'author': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/person/HemingwayErnest1899-1961'}},
   {'book': {'type': 'uri',
     'value': 'http://bnb

In [19]:
# specify columns 

answerCols=['book','title']

In [21]:
for result in results["results"]["bindings"]:
    for ans in answerCols:
        print(result[ans]['value'], end=" ")
    print()

http://bnb.data.bl.uk/id/resource/019676998 The good life according to Hemingway 
http://bnb.data.bl.uk/id/resource/006731265 The essential Hemingway 
http://bnb.data.bl.uk/id/resource/006969792 Death in the afternoon 
http://bnb.data.bl.uk/id/resource/006969811 To have and have not 
http://bnb.data.bl.uk/id/resource/006972503 A farewell to arms 


In [22]:
# function to get data

def printResults(results,ansCols):
    ''' Print the required results column values from the SPARQL query '''
    for result in results["results"]["bindings"]:
        for ans in answerCols:
            print(result[ans]['value'], end=" ")
        print()

In [23]:
printResults(results,answerCols)

http://bnb.data.bl.uk/id/resource/019676998 The good life according to Hemingway 
http://bnb.data.bl.uk/id/resource/006731265 The essential Hemingway 
http://bnb.data.bl.uk/id/resource/006969792 Death in the afternoon 
http://bnb.data.bl.uk/id/resource/006969811 To have and have not 
http://bnb.data.bl.uk/id/resource/006972503 A farewell to arms 


In [32]:
#Let's do a little more wrapping
def runQuery(endpoint,prefix,q):
    ''' Run a SPARQL query with a declared prefix over a specified endpoint '''
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(prefix+q)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def queryResults(endpoint,prefix,q,ansCols):
    ''' Run a SPARQL query with a declared prefix over a specified endpoint and print the required results columns '''
    results=runQuery(endpoint,prefix,q)
    printResults(results,ansCols)

In [33]:
queryResults(endpoint,prefix,q,answerCols)

http://bnb.data.bl.uk/id/resource/019676998 The good life according to Hemingway 
http://bnb.data.bl.uk/id/resource/006731265 The essential Hemingway 
http://bnb.data.bl.uk/id/resource/006969792 Death in the afternoon 
http://bnb.data.bl.uk/id/resource/006969811 To have and have not 
http://bnb.data.bl.uk/id/resource/006972503 A farewell to arms 


In [35]:
# kinda worked

results

{'head': {'link': [], 'vars': ['book', 'title', 'author', 'genre']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'book': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/resource/019676998'},
    'title': {'type': 'literal',
     'value': 'The good life according to Hemingway'},
    'author': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/person/HemingwayErnest1899-1961'}},
   {'book': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/resource/006731265'},
    'title': {'type': 'literal', 'value': 'The essential Hemingway'},
    'author': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/person/HemingwayErnest1899-1961'}},
   {'book': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/resource/006969792'},
    'title': {'type': 'literal', 'value': 'Death in the afternoon'},
    'author': {'type': 'uri',
     'value': 'http://bnb.data.bl.uk/id/person/HemingwayErnest1899-1961'}},
   {'book': {'type': 'uri',
     'value': 'http://bnb

In [26]:
#pandas may have a better way of doing this?!
data=[]
for result in results["results"]["bindings"]:
    tmp={}
    for el in result:
        tmp[el]=result[el]['value']
    data.append(tmp)
    #Note that we lise the type information which we could have used to type the columns in the final dataframe

df = pd.DataFrame(data)
df

,book,title,author
0,http://bnb.data.bl.uk/id/resource/019676998,The good life according to Hemingway,http://bnb.data.bl.uk/id/person/HemingwayErnes...
1,http://bnb.data.bl.uk/id/resource/006731265,The essential Hemingway,http://bnb.data.bl.uk/id/person/HemingwayErnes...
2,http://bnb.data.bl.uk/id/resource/006969792,Death in the afternoon,http://bnb.data.bl.uk/id/person/HemingwayErnes...
3,http://bnb.data.bl.uk/id/resource/006969811,To have and have not,http://bnb.data.bl.uk/id/person/HemingwayErnes...
4,http://bnb.data.bl.uk/id/resource/006972503,A farewell to arms,http://bnb.data.bl.uk/id/person/HemingwayErnes...


In [36]:
#Let's wrap everything up
def dict2df(results):
    ''' Hack a function to flatten the SPARQL query results and return the column values '''
    data=[]
    for result in results["results"]["bindings"]:
        tmp={}
        for el in result:
            tmp[el]=result[el]['value']
        data.append(tmp)

    df = pd.DataFrame(data)
    return df

def dfResults(endpoint,prefix,q):
    ''' Generate a data frame containing the results of running
        a SPARQL query with a declared prefix over a specified endpoint '''
    return dict2df( runQuery( endpoint, prefix, q ) )

In [37]:
dfResults(endpoint,prefix,q)

,book,title,author
0,http://bnb.data.bl.uk/id/resource/019676998,The good life according to Hemingway,http://bnb.data.bl.uk/id/person/HemingwayErnes...
1,http://bnb.data.bl.uk/id/resource/006731265,The essential Hemingway,http://bnb.data.bl.uk/id/person/HemingwayErnes...
2,http://bnb.data.bl.uk/id/resource/006969792,Death in the afternoon,http://bnb.data.bl.uk/id/person/HemingwayErnes...
3,http://bnb.data.bl.uk/id/resource/006969811,To have and have not,http://bnb.data.bl.uk/id/person/HemingwayErnes...
4,http://bnb.data.bl.uk/id/resource/006972503,A farewell to arms,http://bnb.data.bl.uk/id/person/HemingwayErnes...


In [38]:
q='DESCRIBE ?book WHERE { ?book bibo:isbn10 "1857232356" }'
ans=runQuery(endpoint,prefix,q)
ans

C:\Users\creeg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\SPARQLWrapper\Wrapper.py:794: RuntimeWarning: Sending Accept header '*/*' because unexpected returned format 'json' in a 'DESCRIBE' SPARQL query form
  warnings.warn(
C:\Users\creeg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\SPARQLWrapper\Wrapper.py:1179: RuntimeWarning: Format requested was JSON, but N3 (text/turtle; charset=UTF-8) has been returned by the endpoint
  warnings.warn(


b'@prefix owl:\t<http://www.w3.org/2002/07/owl#> .\n@prefix ns1:\t<http://bnb.data.bl.uk/id/resource/> .\nns1:GB9444355\towl:sameAs\t<http://bnb.data.bl.uk/id/resource/012701972> .\n@prefix ns2:\t<http://www.bl.uk/schemas/bibliographic/blterms#> .\n@prefix ns3:\t<http://bnb.data.bl.uk/id/person/> .\nns3:BanksIain1954-2013\tns2:hasCreated\t<http://bnb.data.bl.uk/id/resource/012701972> .\n@prefix rdf:\t<http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix ns5:\t<http://purl.org/dc/terms/> .\n<http://bnb.data.bl.uk/id/resource/012701972>\trdf:type\tns5:BibliographicResource .\n@prefix ns6:\t<http://schema.org/> .\n<http://bnb.data.bl.uk/id/resource/012701972>\trdf:type\tns6:Book .\n@prefix ns7:\t<http://purl.org/ontology/bibo/> .\n<http://bnb.data.bl.uk/id/resource/012701972>\trdf:type\tns7:Book .\n@prefix rdfs:\t<http://www.w3.org/2000/01/rdf-schema#> .\n<http://bnb.data.bl.uk/id/resource/012701972>\trdfs:label\t"Feersum endjinn / Iain Banks" ;\n\towl:sameAs\tns1:GB9444355 ;\n\tns5:cr

In [43]:
q='''
SELECT DISTINCT ?book ?title ?date WHERE {
    #Find books by 'Stephen King':
    ?book dct:creator ?author ;
        dct:title ?title.
    ?author foaf:name "Stephen King".
    
    #Find when they were published:
    ?book blt:publication ?publicationEvent.
    ?publicationEvent event:time ?eventTime.
    ?eventTime rdfs:label ?date.
    
    #Look for books published between 1985 and 1990
    FILTER (?date>="1985" && ?date<"1990")
} ORDER BY ?date 
'''
dfResults(endpoint,prefix,q)

,book,title,date
0,http://bnb.data.bl.uk/id/resource/011306579,The talisman,1985
1,http://bnb.data.bl.uk/id/resource/007388862,Cycle of the werewolf,1985
2,http://bnb.data.bl.uk/id/resource/011324972,Skeleton crew,1985
3,http://bnb.data.bl.uk/id/resource/010690272,It,1986
4,http://bnb.data.bl.uk/id/resource/012634225,Skeleton crew,1986
5,http://bnb.data.bl.uk/id/resource/006727328,The Bachman books : four novels,1986
6,http://bnb.data.bl.uk/id/resource/007389038,The Bachman books,1986
7,http://bnb.data.bl.uk/id/resource/010690904,Misery,1987
8,http://bnb.data.bl.uk/id/resource/012634477,The eyes of the dragon,1987
9,http://bnb.data.bl.uk/id/resource/007389105,It,1987


In [44]:
# convert function to query 

def getBooksByAuthorBetweenDates(author,fromDate,toDate):
    q='''
            SELECT DISTINCT ?book ?title ?date WHERE {{
                #Find books by name:
                ?book dct:creator ?author ;
                    dct:title ?title.
                ?author foaf:name "{0}".

                #Find when they were published:
                ?book blt:publication ?publicationEvent.
                ?publicationEvent event:time ?eventTime.
                ?eventTime rdfs:label ?date.

                #Look for books published between dates
                FILTER (?date>="{1}" && ?date<="{2}")
            }} ORDER BY ?date
        '''.format(author,fromDate,toDate)
    return dfResults(endpoint,prefix,q)

getBooksByAuthorBetweenDates("Stephen King","1985","1987")

,book,title,date
0,http://bnb.data.bl.uk/id/resource/011306579,The talisman,1985
1,http://bnb.data.bl.uk/id/resource/007388862,Cycle of the werewolf,1985
2,http://bnb.data.bl.uk/id/resource/011324972,Skeleton crew,1985
3,http://bnb.data.bl.uk/id/resource/010690272,It,1986
4,http://bnb.data.bl.uk/id/resource/012634225,Skeleton crew,1986
5,http://bnb.data.bl.uk/id/resource/006727328,The Bachman books : four novels,1986
6,http://bnb.data.bl.uk/id/resource/007389038,The Bachman books,1986
7,http://bnb.data.bl.uk/id/resource/010690904,Misery,1987
8,http://bnb.data.bl.uk/id/resource/012634477,The eyes of the dragon,1987
9,http://bnb.data.bl.uk/id/resource/007389105,It,1987
